In [1]:
import plotnine as p9
import ee

In [2]:
ee.Initialize()

In [10]:
# @st.cache
def get_image_near_point(im_collection_id, im_date,  bands_rgb, latitude, longitude, buffer_px, 
                        return_geopandas = False):
    
        
    start_datetime = datetime.strptime(im_date,'%Y-%m-%d')
    end_date = datetime.strftime(start_datetime + timedelta(days = 1), '%Y-%m-%d')
    
    try:
        pt = ee.Geometry.Point([longitude, latitude])
    except:
        ee.Initialize()
        pt = ee.Geometry.Point([longitude, latitude])
        
    # pt_bbox = pt.buffer(buffer_m, 1).bounds()
    ic = ee.ImageCollection(im_collection_id).filterBounds(pt).filterDate(im_date, end_date)
    # ic = ic.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 0.25))
    im = ic.first().select(bands_rgb)

    if return_geopandas:
        im = im.addBands(im.pixelCoordinates(im.projection()))
    
    # buffer_px = 10
    # generate kernel
    imdim = (buffer_px * 2) + 1
    kernel_list = ee.List.repeat(1, imdim)
    kernel_lists = ee.List.repeat(kernel_list, imdim)
    kernel = ee.Kernel.fixed(imdim, imdim, kernel_lists, 
                             x = buffer_px + 1, y = buffer_px + 1)
    
    
    im_eearray = im.neighborhoodToArray(kernel)
    
    
    # sample the region and return array from ee to python
    im_dict = im_eearray.reduceRegion(ee.Reducer.first(), geometry = pt, scale = 10).getInfo()
    # # old, with bounding box:
    # im_dict = im.sampleRectangle(region = pt_bbox, properties = []).getInfo()
    # im_props = im_dict['properties']

    # len(im_dict['properties'])
    im_props = im_dict
    im_props_keys = list(im_props.keys())

    if return_geopandas:
        df = pd.DataFrame()
        for i in range(len(im_props)):
            colname = im_props_keys[i]
            data = np.array(im_props[colname]).flatten()
            df[colname] = data

        im_projection = im.projection().getInfo()
        # convert to geopandas
        gdf = gpd.GeoDataFrame(df, 
                 geometry = gpd.points_from_xy(df.x, df.y),
                 crs = im_projection['crs'])
        
        return_val = gdf


    else:
        # extract each band separately
        Bred = np.expand_dims(np.array(im_props[bands_rgb[0]]), 2)
        Bgreen = np.expand_dims(np.array(im_props[bands_rgb[1]]), 2)
        Bblue = np.expand_dims(np.array(im_props[bands_rgb[2]]), 2)

        im_array_rgb = np.concatenate((Bred, Bgreen, Bblue), axis = 2)
        return_val = im_array_rgb
    
    return return_val



# @st.cache
def plot_array_image(im_array):
    xcenter = math.floor(im_array.shape[0] / 2)
    ycenter = math.floor(im_array.shape[1] / 2)
    
    maxval = 5000
    
    im_array[im_array > maxval] = maxval
    
    arrow_spacing = 1
    # Scale the data to [0, 255] to show as an RGB image.
    rgb_img_test = (255*((im_array - 0)/maxval)).astype('uint8')
    plt.figure(figsize = (5,5),dpi = 100)
    plt.axis('off')
    plt.imshow(rgb_img_test)
    plt.plot(xcenter, ycenter - arrow_spacing, marker = 'v', color = 'white')
    plt.plot(xcenter, ycenter + arrow_spacing,marker = '^', color = 'white')
    plt.plot(xcenter + arrow_spacing, ycenter, marker = '<', color = 'white')
    plt.plot(xcenter - arrow_spacing, ycenter,marker = '>', color = 'white')
    return plt

SyntaxError: unexpected EOF while parsing (595123356.py, line 3)